In [ ]:
"""
포즈 감지 및 분류를 위한 딥러닝 모델 구현
주요 기능: 실시간 포즈 감지, 다중 모델 앙상블, 포즈 분류
작성자: 오승진
마지막 수정: 2024
"""

# 기본 라이브러리 임포트
import collections
import cv2
import mediapipe as mp
import numpy as np
import time
from PIL import ImageFont, ImageDraw, Image
from collections import deque
import threading
from queue import Queue
import pandas as pd
import os
import sys

# 딥러닝 관련 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO

class PoseTransformer(nn.Module):
    """
    Transformer 아키텍처 기반의 포즈 분류 모델
    
    주요 특징:
    - 17개의 신체 키포인트를 처리하는 Transformer 기반 아키텍처
    - 위치 임베딩을 통한 공간 정보 인코딩
    - 멀티헤드 어텐션 메커니즘 활용
    - 드롭아웃을 통한 과적합 방지
    
    Parameters:
        input_dim (int): 입력 데이터의 차원 (기본값: 34, 17개 키포인트 x 2좌표)
        num_classes (int): 분류할 포즈 클래스의 수 (기본값: 20)
        num_heads (int): Transformer의 어텐션 헤드 수 (기본값: 8)
        dim_feedforward (int): 피드포워드 네트워크의 은닉층 차원 (기본값: 512)
        num_layers (int): Transformer 인코더 레이어의 수 (기본값: 4)
    
    입력 데이터 형식:
        - (batch_size, 34) 크기의 텐서
        - 각 키포인트는 (x, y) 좌표로 표현
    
    출력:
        - (batch_size, num_classes) 크기의 텐서
        - 각 클래스에 대한 확률 분포
    """
    def __init__(self, input_dim=34, num_classes=20, num_heads=8, dim_feedforward=512, num_layers=4):
        super().__init__()
        
        # 2D 좌표를 고차원 벡터로 투영하는 레이어
        self.input_projection = nn.Linear(2, 128)
        
        # 17개 키포인트에 대한 위치 임베딩
        # 학습 가능한 파라미터로, 각 키포인트의 공간적 위치 정보를 인코딩
        self.positional_embedding = nn.Parameter(torch.randn(17, 128))
        
        # Transformer 인코더 레이어 설정
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=128,          # 모델의 기본 차원
            nhead=num_heads,      # 병렬 어텐션 헤드의 수
            dim_feedforward=dim_feedforward,  # FFN의 은닉층 크기
            dropout=0.1,          # 드롭아웃 비율
            batch_first=True      # 배치 차원을 첫 번째로 설정
        )
        
        # 여러 개의 인코더 레이어를 쌓아 Transformer 구성
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 최종 분류를 위한 완전연결 레이어들
        self.classifier = nn.Sequential(
            nn.Linear(128 * 17, 512),  # 특징을 512차원으로 확장
            nn.ReLU(),                 # 비선형성 추가
            nn.Dropout(0.2),           # 과적합 방지
            nn.Linear(512, 256),       # 차원 축소
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)  # 최종 클래스 수만큼 출력
        )

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 입력 텐서를 (배치크기, 17, 2) 형태로 재구성
        2. 2D 좌표를 128차원 벡터로 투영
        3. 위치 임베딩 정보 추가
        4. Transformer를 통한 특징 추출
        5. 분류기를 통한 최종 예측 생성
        
        Args:
            x (torch.Tensor): (배치크기, 34) 크기의 입력 텐서
                            34 = 17개 키포인트 * 2(x,y 좌표)
        
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 클래스 예측값
        """
        x = x.view(-1, 17, 2)        # 키포인트 좌표 형태로 재구성
        x = self.input_projection(x)  # 고차원 특징 공간으로 투영
        x = x + self.positional_embedding  # 위치 정보 추가
        x = self.transformer(x)       # Transformer 처리
        x = x.reshape(x.size(0), -1)  # 분류기 입력을 위해 평탄화
        x = self.classifier(x)        # 최종 분류
        return x
    
class PoseMLP(nn.Module):
    """
    다층 퍼셉트론(MLP) 기반의 포즈 분류 모델
    
    주요 특징:
    - 잔차 연결(Residual connections)을 포함한 심층 신경망 구조
    - BatchNormalization을 통한 학습 안정화
    - 드롭아웃을 통한 과적합 방지
    - 4개의 잔차 블록으로 구성된 깊은 아키텍처
    
    Parameters:
        input_dim (int): 입력 데이터의 차원 (기본값: 34, 17개 키포인트 x 2좌표)
        num_classes (int): 분류할 포즈 클래스의 수 (기본값: 20)
    """
    def __init__(self, input_dim=34, num_classes=20):
        super().__init__()
        
        # 첫 번째 특징 추출 블록
        self.block1 = nn.Sequential(
            nn.Linear(input_dim, 512),    # 입력을 512차원으로 확장
            nn.BatchNorm1d(512),          # 배치 정규화
            nn.ReLU(),                    # 비선형성 추가
            nn.Dropout(0.3)               # 30% 드롭아웃
        )
        
        # 4개의 잔차 블록 생성
        self.res_blocks = nn.ModuleList([
            self._make_res_block(512, 512) for _ in range(4)
        ])
        
        # 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),          # 차원 축소
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)    # 최종 클래스 예측
        )

    def _make_res_block(self, in_dim, out_dim):
        """
        잔차 블록을 생성하는 헬퍼 메서드
        
        구조:
        입력 -> Linear -> BatchNorm -> ReLU -> Dropout -> Linear -> BatchNorm -> (+) 입력 -> ReLU
        
        Args:
            in_dim (int): 입력 차원
            out_dim (int): 출력 차원
            
        Returns:
            nn.Sequential: 구성된 잔차 블록
        """
        return nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.BatchNorm1d(out_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(out_dim, out_dim),
            nn.BatchNorm1d(out_dim)
        )

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 첫 번째 특징 추출 블록 통과
        2. 각 잔차 블록을 순차적으로 통과하며 잔차 연결 적용
        3. 최종 분류기를 통한 예측 생성
        
        Args:
            x (torch.Tensor): (배치크기, input_dim) 크기의 입력 텐서
            
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 클래스 예측값
        """
        x = self.block1(x)
        for res_block in self.res_blocks:
            identity = x                    # 잔차 연결을 위해 입력 저장
            x = res_block(x)                # 잔차 블록 통과
            x = F.relu(x + identity)        # 잔차 연결 및 활성화
        x = self.classifier(x)              # 최종 분류
        return x

class PoseGRU(nn.Module):
    """
    GRU(Gated Recurrent Unit) 기반의 포즈 분류 모델
    
    주요 특징:
    - 양방향 GRU를 통한 시계열 데이터 처리
    - 키포인트 간의 시퀀스적 관계 학습
    - 공간적 임베딩을 통한 특징 추출
    
    Parameters:
        input_dim (int): 입력 데이터의 차원 (기본값: 34)
        num_classes (int): 분류할 포즈 클래스의 수 (기본값: 20)
        hidden_dim (int): GRU의 은닉 상태 차원 (기본값: 256)
    """
    def __init__(self, input_dim=34, num_classes=20, hidden_dim=256):
        super().__init__()
        
        # 공간적 특징을 추출하는 임베딩 레이어
        self.spatial_embedding = nn.Linear(2, hidden_dim)
        
        # 양방향 GRU 레이어
        self.gru = nn.GRU(
            input_size=hidden_dim,         # 입력 크기
            hidden_size=hidden_dim,        # 은닉 상태 크기
            num_layers=3,                  # GRU 레이어 수
            batch_first=True,              # 배치 차원을 첫 번째로 설정
            dropout=0.2,                   # 레이어 간 드롭아웃
            bidirectional=True             # 양방향 처리
        )
        
        # 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),  # 양방향이므로 *2
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 입력을 (배치크기, 17, 2) 형태로 재구성
        2. 공간적 임베딩 적용
        3. GRU를 통한 시퀀스 처리
        4. 최종 시점의 은닉 상태를 사용하여 분류
        
        Args:
            x (torch.Tensor): (배치크기, input_dim) 크기의 입력 텐서
            
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 클래스 예측값
        """
        x = x.view(-1, 17, 2)              # 키포인트 시퀀스 형태로 재구성
        x = self.spatial_embedding(x)       # 공간적 특징 추출
        x, _ = self.gru(x)                 # GRU 처리
        x = x[:, -1, :]                    # 마지막 시점의 출력 사용
        x = self.classifier(x)              # 최종 분류
        return x
    
class PoseCNN(nn.Module):
    """
    CNN(Convolutional Neural Network) 기반의 포즈 분류 모델
    
    주요 특징:
    - 1D 컨볼루션을 사용한 키포인트 시퀀스 처리
    - BatchNormalization을 통한 학습 안정화
    - 계층적 특징 추출을 통한 공간적 관계 학습
    
    구조적 특징:
    - 3개의 연속된 컨볼루션 레이어
    - 채널 수를 점진적으로 증가 (64 -> 128 -> 256)
    - 완전연결 레이어를 통한 최종 분류
    
    Parameters:
        input_dim (int): 입력 데이터의 차원 (기본값: 34, 17개 키포인트 x 2좌표)
        num_classes (int): 분류할 포즈 클래스의 수 (기본값: 20)
    """
    def __init__(self, input_dim=34, num_classes=20):
        super().__init__()
        
        # 공간적 특징을 추출하는 1D 컨볼루션 레이어들
        self.spatial_conv = nn.Sequential(
            # 첫 번째 컨볼루션 블록: 2 -> 64 채널
            nn.Conv1d(2, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
            # 두 번째 컨볼루션 블록: 64 -> 128 채널
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
            # 세 번째 컨볼루션 블록: 128 -> 256 채널
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        # 분류기 네트워크
        self.classifier = nn.Sequential(
            nn.Linear(256 * 17, 512),  # 특징 맵을 평탄화하여 전결합층으로 전달
            nn.ReLU(),
            nn.Dropout(0.3),           # 과적합 방지
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 입력을 [batch, channels(x,y), keypoints] 형태로 재구성
        2. 1D 컨볼루션을 통한 특징 추출
        3. 추출된 특징을 평탄화하여 분류기에 전달
        
        Args:
            x (torch.Tensor): (배치크기, input_dim) 크기의 입력 텐서
            
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 클래스 예측값
        """
        x = x.view(-1, 2, 17)         # [batch, channels(x,y), keypoints] 형태로 변환
        x = self.spatial_conv(x)       # 컨볼루션 연산 적용
        x = x.reshape(x.size(0), -1)   # 특징 맵 평탄화
        x = self.classifier(x)         # 최종 분류
        return x

class EnhancedPoseViT(nn.Module):
    """
    Vision Transformer(ViT) 기반의 향상된 포즈 분류 모델
    
    주요 특징:
    - 키포인트를 토큰으로 처리하는 Transformer 구조
    - 글로벌 어텐션 풀링을 통한 특징 집계
    - LayerNorm을 통한 정규화
    - 어텐션 기반의 가중치 풀링
    
    아키텍처 특징:
    - 초기 임베딩 레이어
    - 멀티헤드 셀프 어텐션 기반 Transformer
    - 어텐션 기반 풀링 메커니즘
    - 계층적 분류기
    
    Parameters:
        input_dim (int): 입력 데이터의 차원 (기본값: 34)
        num_classes (int): 분류할 클래스 수 (기본값: 20)
    """
    def __init__(self, input_dim=34, num_classes=20):
        super().__init__()
        
        # 초기 임베딩 레이어
        self.embedding = nn.Linear(2, 128)
        
        # Transformer 인코더 설정
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=128,           # 모델 차원
            nhead=4,               # 어텐션 헤드 수
            dim_feedforward=256,   # 피드포워드 네트워크 차원
            dropout=0.1,           # 드롭아웃 비율
            batch_first=True       # 배치 우선 처리
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=3          # Transformer 레이어 수
        )
        
        # 글로벌 어텐션 풀링
        self.attention_pooling = nn.Sequential(
            nn.Linear(128, 64),    # 차원 축소
            nn.Tanh(),             # 활성화 함수
            nn.Linear(64, 1)       # 어텐션 스코어 생성
        )
        
        # 분류 헤드
        self.classifier = nn.Sequential(
            nn.Linear(128, 256),
            nn.LayerNorm(256),     # 레이어 정규화
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 입력을 키포인트 시퀀스로 재구성
        2. 키포인트 임베딩
        3. Transformer를 통한 특징 추출
        4. 어텐션 기반 풀링으로 특징 집계
        5. 최종 분류 수행
        
        Args:
            x (torch.Tensor): (배치크기, 34) 크기의 입력 텐서
            
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 클래스 예측값
        """
        x = x.view(-1, 17, 2)          # 키포인트 시퀀스로 재구성
        x = self.embedding(x)           # 임베딩 적용
        x = self.transformer(x)         # Transformer 처리
        
        # 어텐션 가중치 계산 및 적용
        attn_weights = self.attention_pooling(x)
        attn_weights = F.softmax(attn_weights, dim=1)  # 소프트맥스로 정규화
        x = (x * attn_weights).sum(dim=1)  # 가중치 적용 및 합산
        
        x = self.classifier(x)          # 최종 분류
        return x
    
class StackingModel(nn.Module):
    """
    앙상블 스태킹 기반의 포즈 분류 모델
    
    주요 특징:
    - 5개의 기본 모델(Transformer, MLP, GRU, CNN, ViT)을 결합
    - 각 모델의 예측에 대한 학습 가능한 가중치 적용
    - 메타 학습을 통한 최적의 모델 조합 학습
    
    앙상블 구조:
    1. 각 기본 모델이 독립적으로 예측 수행
    2. 예측값들을 가중치와 함께 결합
    3. 메타 특징 추출기를 통한 고차원 패턴 학습
    4. 최종 분류기를 통한 예측
    
    Parameters:
        num_classes (int): 분류할 클래스 수 (기본값: 20)
        num_base_models (int): 기본 모델의 수 (기본값: 5)
    """
    def __init__(self, num_classes=20, num_base_models=5):
        super().__init__()
        
        # 기본 모델들 초기화
        self.transformer = PoseTransformer()    # Transformer 기반 모델
        self.mlp = PoseMLP()                   # MLP 기반 모델
        self.gru = PoseGRU()                   # GRU 기반 모델
        self.cnn = PoseCNN()                   # CNN 기반 모델
        self.attention = EnhancedPoseViT()     # ViT 기반 모델
        
        # 메타 특징 추출기
        # 각 모델의 예측을 결합하고 고차원 패턴을 학습
        self.meta_features = nn.Sequential(
            nn.Linear(num_classes * 5, 512),  # 모든 모델의 예측을 연결
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # 최종 분류기
        # 메타 특징을 기반으로 최종 예측 생성
        self.final_classifier = nn.Sequential(
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)  # 로그 소프트맥스 활성화
        )
        
        # 각 모델의 가중치 파라미터
        # 학습을 통해 각 모델의 중요도를 자동으로 결정
        self.model_weights = nn.Parameter(torch.ones(5))

    def forward(self, x):
        """
        순전파 연산을 수행하는 메서드
        
        처리 단계:
        1. 각 기본 모델에서 독립적으로 예측 수행
        2. 예측값에 소프트맥스 적용하여 확률 분포 생성
        3. 모델 가중치를 정규화하고 예측값에 적용
        4. 가중치가 적용된 예측값들을 결합
        5. 메타 특징 추출 및 최종 예측 생성
        
        Args:
            x (torch.Tensor): (배치크기, input_dim) 크기의 입력 텐서
            
        Returns:
            torch.Tensor: (배치크기, num_classes) 크기의 최종 클래스 예측값
        """
        # 각 모델의 예측 수행
        trans_out = self.transformer(x)
        mlp_out = self.mlp(x)
        gru_out = self.gru(x)
        cnn_out = self.cnn(x)
        attn_out = self.attention(x)
        
        # 각 예측값을 확률 분포로 변환
        trans_prob = F.softmax(trans_out, dim=1)
        mlp_prob = F.softmax(mlp_out, dim=1)
        gru_prob = F.softmax(gru_out, dim=1)
        cnn_prob = F.softmax(cnn_out, dim=1)
        attn_prob = F.softmax(attn_out, dim=1)
        
        # 모델 가중치 정규화
        weights = F.softmax(self.model_weights, dim=0)
        
        # 가중치를 적용하여 예측값 결합
        stacked_features = torch.cat([
            trans_prob * weights[0],
            mlp_prob * weights[1],
            gru_prob * weights[2],
            cnn_prob * weights[3],
            attn_prob * weights[4]
        ], dim=1)
        
        # 메타 특징 추출
        meta_features = self.meta_features(stacked_features)
        
        # 최종 예측
        output = self.final_classifier(meta_features)
        return output

# 클래스 매핑 정의
class_mapping = {
    """
    포즈 클래스 ID와 해당하는 한글 레이블 매핑
    
    각 숫자 키는 특정 포즈를 나타내며,
    값은 해당 포즈의 한글 설명을 포함
    """
    0: "A포즈",
    1: "I포즈",
    2: "T포즈",
    3: "계단 오르기",
    4: "공을 던지려고 힘을 주는 자세",
    5: "기지개",
    6: "달리기(전력질주)",
    7: "뒷짐",
    8: "막대를 양손으로 잡고 골반 뒤쪽으로 쭉 뻗은 자세",
    9: "머리 뒤 깍지를 낀 자세",
    10: "몸을 앞으로 숙인 자세",
    11: "발레",
    12: "벽에 기대어 신발 신기",
    13: "의자에 앉은 자세",
    14: "조깅",
    15: "통화하는 자세",
    16: "팔짱",
    17: "한 손과 반대편 발을 들며 신난 자세",
    18: "한 다리 올리고 편하게 앉은 자세",
    19: "허리 회전을 최대로 한 자세"
}

def setup_device():
    """
    GPU/CPU 사용 설정을 처리하는 함수
    
    주요 기능:
    - CUDA GPU 사용 가능 여부 확인
    - GPU 사용 가능시 CUDA 최적화 설정 적용
    - GPU 메모리 및 디바이스 정보 출력
    
    Returns:
        torch.device: 사용할 디바이스 객체
    """
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # CUDA 성능 최적화 설정
        torch.backends.cudnn.benchmark = True      # 자동 벤치마킹으로 최적의 알고리즘 선택
        torch.backends.cudnn.deterministic = False # 성능 향상을 위해 결정성 비활성화
        
        # GPU 정보 출력
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # GB 단위 변환
        print(f"Using GPU: {gpu_name}")
        print(f"Total GPU Memory: {gpu_memory:.2f} GB")
    else:
        device = torch.device('cpu')
        print("GPU not available, using CPU")
    return device

class PoseTransformer(nn.Module):
    def __init__(self, input_dim=34, num_classes=20, num_heads=8, dim_feedforward=512, num_layers=4):
        """
        포즈 분류를 위한 Transformer 모델 초기화
        
        주요 컴포넌트:
        - MediaPipe 포즈 추정기
        - YOLO 객체 탐지기
        - 키포인트 처리 및 안정화 메커니즘
        - 다양한 키포인트 매핑 및 시각화 설정
        """
        super().__init__()
        self.device = setup_device()  # GPU/CPU 설정
        
        # MediaPipe 포즈 추정기 초기화
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,        # 비디오 스트림 모드
            model_complexity=1,             # 중간 수준의 복잡도
            smooth_landmarks=True,          # 키포인트 스무딩 활성화
            min_detection_confidence=0.5,   # 최소 탐지 신뢰도
            min_tracking_confidence=0.5     # 최소 추적 신뢰도
        )

        # YOLO 키포인트 인덱스 매핑 정의
        self.yolo_keypoint_indices = {
            'nose': 0,
            'left_eye': 1,
            'right_eye': 2,
            'left_ear': 3,
            'right_ear': 4,
            # ... (기타 키포인트)
        }

        # 키포인트 열 매핑 정의 (정규화용)
        self.column_mapping = {
            'nose': ['Nose_x', 'Nose_y'],
            'left_eye': ['Left_Eye_x', 'Left_Eye_y'],
            # ... (기타 매핑)
        }

        # MediaPipe 키포인트 매핑
        self.mediapipe_keypoints = {
            self.mp_pose.PoseLandmark.NOSE: 'nose',
            self.mp_pose.PoseLandmark.LEFT_EYE: 'left_eye',
            # ... (기타 매핑)
        }

        # 스켈레톤 연결 정의
        self.skeleton_connections = [
            ('nose', 'left_eye'), ('nose', 'right_eye'),
            ('left_eye', 'left_ear'), ('right_eye', 'right_ear'),
            # ... (기타 연결)
        ]

        # 키포인트 색상 설정
        self.keypoint_colors = {
            # YOLO로 검출한 키포인트 (빨간색)
            'nose': (0, 0, 255),
            'left_eye': (0, 0, 255),
            # MediaPipe로 검출한 키포인트 (파란색)
            'left_shoulder': (255, 0, 0),
            'right_shoulder': (255, 0, 0),
            # ... (기타 색상)
        }

        # YOLO 모델 설정
        self.yolo_pose = YOLO(yolo_pose_path)
        if self.device.type == 'cuda':
            self.yolo_pose.to(self.device)
        self.yolo_pose.conf = 0.15  # 신뢰도 임계값
        self.yolo_pose.iou = 0.35   # IOU 임계값

        # 키포인트 안정화 설정
        self.keypoint_buffer = collections.deque(maxlen=5)  # 최근 5프레임 저장
        self.stability_threshold = 100       # 키포인트 안정성 임계값
        self.movement_threshold = 0.5        # 움직임 감지 임계값
        self.smoothing_factor = 0.7         # 스무딩 계수
        self.previous_keypoints = None       # 이전 프레임 키포인트
        self.lost_tracking_frames = 0        # 추적 실패 프레임 카운트
        self.frame_width = 640              # 프레임 너비
        self.frame_height = 480             # 프레임 높이

        # 베이스 모델과 스태킹 모델 로드
        self.base_models = self.load_base_models(model_paths)
        self.stacking_model = self.load_stacking_model(model_paths['stacking'])

    def _combination_keypoints(self, image):
        """
        MediaPipe와 YOLO 모델의 키포인트를 결합하는 메서드
        
        주요 기능:
        - 여러 사람의 키포인트 동시 처리
        - YOLO와 MediaPipe의 장점을 결합한 하이브리드 접근
        - 신뢰도 기반의 키포인트 선택
        
        처리 단계:
        1. 이미지 전처리 및 변환
        2. YOLO를 통한 키포인트 및 바운딩 박스 검출
        3. MediaPipe를 통한 상세 키포인트 검출
        4. 두 모델의 결과 결합
        
        Args:
            image (np.ndarray): 처리할 BGR 이미지
            
        Returns:
            list: 각 감지된 사람의 키포인트 정보를 담은 딕셔너리 리스트
        """
        try:
            height, width = image.shape[:2]
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            all_person_keypoints = []  # 여러 사람의 키포인트 저장용

            # YOLO 포즈 검출 수행
            yolo_results = self.yolo_pose(image_rgb)
            person_boxes = self.yolo(image_rgb)
            
            if len(yolo_results) > 0:
                # 각 감지된 사람에 대한 처리
                for person_idx in range(len(yolo_results[0].keypoints)):
                    combination_keypoints = {}
                    
                    # YOLO 키포인트 처리
                    if hasattr(yolo_results[0], 'keypoints'):
                        kpts = yolo_results[0].keypoints[person_idx]
                        if hasattr(kpts, 'data') and len(kpts.data) > 0:
                            yolo_kpts = kpts.data.cpu().numpy()[0]
                            for idx, (key, _) in enumerate(self.yolo_keypoint_indices.items()):
                                if idx < len(yolo_kpts):
                                    x, y, conf = yolo_kpts[idx]
                                    if conf > 0.2:  # 신뢰도 임계값
                                        combination_keypoints[key] = [int(x), int(y)]

                    # MediaPipe 처리
                    mp_results = self.pose.process(image_rgb)
                    if mp_results.pose_landmarks:
                        landmarks = mp_results.pose_landmarks.landmark
                        for landmark_id, name in self.mediapipe_keypoints.items():
                            landmark = landmarks[landmark_id.value]
                            if landmark.visibility > 0.3:  # 가시성 임계값
                                x = int(landmark.x * width)
                                y = int(landmark.y * height)
                                # 상체 키포인트는 MediaPipe 우선
                                if name in ['left_shoulder', 'right_shoulder',
                                        'left_elbow', 'right_elbow',
                                        'left_wrist', 'right_wrist']:
                                    combination_keypoints[name] = [x, y]

                    # 바운딩 박스 정보 추가
                    if len(person_boxes) > person_idx:
                        box = person_boxes[0].boxes.data[person_idx]
                        x1, y1, x2, y2, conf, _ = box.cpu().numpy()
                        combination_keypoints['bbox'] = [
                            int(x1), int(y1), int(x2), int(y2), float(conf)
                        ]

                    # 유효성 검증 및 추가
                    if len(combination_keypoints) > 5:  # 최소 키포인트 수 검증
                        all_person_keypoints.append(combination_keypoints)

            return all_person_keypoints

        except Exception as e:
            print(f"키포인트 결합 중 오류 발생: {str(e)}")
            return None

    def smooth_keypoints(self, current_keypoints):
        """
        키포인트 움직임을 부드럽게 하는 스무딩 처리 메서드
        
        주요 기능:
        - 시간적 필터링을 통한 키포인트 안정화
        - 지수 이동 평균을 사용한 스무딩
        - 급격한 변화 감지 및 보정
        
        처리 단계:
        1. 이전 프레임들의 유효한 키포인트 확인
        2. 지수 이동 평균 계산
        3. 스무딩된 좌표 계산
        
        Args:
            current_keypoints (dict): 현재 프레임의 키포인트
            
        Returns:
            dict: 스무딩된 키포인트
        """
        if not self.keypoint_buffer:
            return current_keypoints

        smoothed_keypoints = {}
        for key in current_keypoints:
            if current_keypoints[key] == [0, 0]:
                continue

            # 버퍼에서 유효한 이전 키포인트 추출
            valid_points = [
                buff[key] for buff in self.keypoint_buffer
                if key in buff and buff[key] != [0, 0]
            ]

            if valid_points:
                # 지수 이동 평균 계산
                curr_x, curr_y = current_keypoints[key]
                prev_x, prev_y = valid_points[-1]

                # 스무딩 적용
                smoothed_x = self.smoothing_factor * curr_x + (1 - self.smoothing_factor) * prev_x
                smoothed_y = self.smoothing_factor * curr_y + (1 - self.smoothing_factor) * prev_y

                smoothed_keypoints[key] = [int(smoothed_x), int(smoothed_y)]
            else:
                smoothed_keypoints[key] = current_keypoints[key]

        return smoothed_keypoints
    
    def extract_keypoints(self, image):
        """
        이미지에서 키포인트를 추출하는 메인 메서드
        
        주요 기능:
        - MediaPipe와 YOLO 모델을 통합적으로 사용
        - 해부학적 제약 조건 적용
        - 누락된 키포인트 추정
        - 키포인트 안정성 검증
        
        처리 과정:
        1. 입력 이미지 유효성 검사
        2. 모델 기반 키포인트 추출
        3. 해부학적 제약 조건 적용
        4. 누락된 키포인트 보간
        
        Args:
            image (np.ndarray): 처리할 입력 이미지
            
        Returns:
            dict: 처리된 키포인트 정보 또는 None (실패 시)
        """
        try:
            if not isinstance(image, np.ndarray) or image.size == 0:
                return None

            # 키포인트 추출 및 결합
            keypoints = self._combination_keypoints(image)
            if keypoints is None:
                return None

            # 해부학적 제약 조건 적용
            keypoints = self._apply_anatomical_constraints(keypoints, image.shape[0], image.shape[1])

            # 누락된 키포인트 추정
            keypoints = self.estimate_hidden_keypoints(keypoints)

            return keypoints

        except Exception as e:
            print(f"키포인트 추출 중 오류 발생: {str(e)}")
            return None

    def _apply_anatomical_constraints(self, keypoints, height, width):
        """
        해부학적 제약 조건을 적용하여 키포인트의 유효성을 검증하고 보정하는 메서드
        
        주요 기능:
        - 얼굴 부위 키포인트 제약 적용
        - 다리 관절 체인 검증
        - 비현실적인 관절 위치 보정
        
        제약 조건:
        1. 얼굴 키포인트 간 최대 거리 제한
        2. 관절 간 최대 거리 제한
        3. 해부학적으로 가능한 관절 각도 범위 검증
        
        Args:
            keypoints (dict): 입력 키포인트
            height (int): 이미지 높이
            width (int): 이미지 너비
            
        Returns:
            dict: 제약 조건이 적용된 키포인트
        """
        constrained_points = keypoints.copy()

        # 얼굴 키포인트 제약
        if 'nose' in constrained_points:
            nose_x, nose_y = constrained_points['nose']
            face_radius = width * 0.1  # 얼굴 크기 제한

            # 얼굴 부위 키포인트 위치 제한
            for name in ['left_eye', 'right_eye', 'left_ear', 'right_ear']:
                if name in constrained_points:
                    x, y = constrained_points[name]
                    dist = np.sqrt((x - nose_x) ** 2 + (y - nose_y) ** 2)

                    if dist > face_radius:
                        # 최대 거리 제한 적용
                        angle = np.arctan2(y - nose_y, x - nose_x)
                        constrained_points[name] = [
                            int(nose_x + face_radius * np.cos(angle)),
                            int(nose_y + face_radius * np.sin(angle))
                        ]

        # 다리 키포인트 제약
        for side in ['left', 'right']:
            hip_name = f'{side}_hip'
            knee_name = f'{side}_knee'
            ankle_name = f'{side}_ankle'

            if all(name in constrained_points for name in [hip_name, knee_name, ankle_name]):
                hip = constrained_points[hip_name]
                knee = constrained_points[knee_name]
                ankle = constrained_points[ankle_name]

                # 무릎-엉덩이 거리 제한
                knee_hip_dist = np.sqrt((knee[0] - hip[0]) ** 2 + (knee[1] - hip[1]) ** 2)
                max_knee_dist = height * 0.3

                if knee_hip_dist > max_knee_dist:
                    angle = np.arctan2(knee[1] - hip[1], knee[0] - hip[0])
                    constrained_points[knee_name] = [
                        int(hip[0] + max_knee_dist * np.cos(angle)),
                        int(hip[1] + max_knee_dist * np.sin(angle))
                    ]

                # 발목-무릎 거리 제한
                ankle_knee_dist = np.sqrt((ankle[0] - knee[0]) ** 2 + (ankle[1] - knee[1]) ** 2)
                max_ankle_dist = height * 0.3

                if ankle_knee_dist > max_ankle_dist:
                    angle = np.arctan2(ankle[1] - knee[1], ankle[0] - knee[0])
                    constrained_points[ankle_name] = [
                        int(knee[0] + max_ankle_dist * np.cos(angle)),
                        int(knee[1] + max_ankle_dist * np.sin(angle))
                    ]

        return constrained_points
    
    def estimate_hidden_keypoints(self, keypoints):
        """
        숨겨지거나 누락된 키포인트를 추정하는 메서드
        
        주요 기능:
        - 이전 프레임 정보를 활용한 키포인트 추정
        - 움직임 기반 예측 및 보정
        - 급격한 움직임 감지 및 처리
        - 시간적 일관성 유지
        
        처리 단계:
        1. 현재 키포인트 유효성 검사
        2. 이전 프레임 데이터 활용
        3. 움직임 패턴 분석
        4. 누락된 키포인트 추정
        
        Args:
            keypoints (dict): 현재 프레임의 키포인트 데이터
            
        Returns:
            dict: 추정된 키포인트가 포함된 완성된 데이터
        """
        if keypoints is None:
            return self.previous_keypoints

        estimated_keypoints = keypoints.copy()

        # 움직임이 큰 경우의 처리
        if self.previous_keypoints:
            for key in keypoints:
                curr_x, curr_y = keypoints[key]
                if key in self.previous_keypoints:
                    prev_x, prev_y = self.previous_keypoints[key]

                    # 현재 위치가 유효하지 않은 경우 처리
                    if (curr_x == 0 and curr_y == 0) or \
                            curr_x < 0 or curr_x > self.frame_width or \
                            curr_y < 0 or curr_y > self.frame_height:

                        # 이전 프레임들의 데이터를 활용한 추정
                        if len(self.keypoint_buffer) >= 2:
                            older_x, older_y = self.keypoint_buffer[-2].get(key, (prev_x, prev_y))
                            if older_x != 0 and older_y != 0:
                                # 이동 방향과 속도를 고려한 예측
                                dx = prev_x - older_x
                                dy = prev_y - older_y
                                estimated_x = int(prev_x + dx * self.movement_threshold)
                                estimated_y = int(prev_y + dy * self.movement_threshold)
                                
                                # 예측값이 화면 범위 내에 있는지 확인
                                estimated_x = max(0, min(estimated_x, self.frame_width))
                                estimated_y = max(0, min(estimated_y, self.frame_height))
                                
                                estimated_keypoints[key] = [estimated_x, estimated_y]
                                continue

                            estimated_keypoints[key] = self.previous_keypoints[key]
                    else:
                        # 급격한 움직임 감지 및 보정
                        distance = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2)
                        if distance > self.stability_threshold:
                            if len(self.keypoint_buffer) > 0:
                                valid_points = [
                                    buff[key] for buff in self.keypoint_buffer
                                    if key in buff and buff[key] != [0, 0]
                                ]
                                if valid_points:
                                    recent_points = valid_points[-2:]
                                    if len(recent_points) >= 2:
                                        dx = recent_points[1][0] - recent_points[0][0]
                                        dy = recent_points[1][1] - recent_points[0][1]
                                        estimated_keypoints[key] = [
                                            int(recent_points[1][0] + dx * 0.5),
                                            int(recent_points[1][1] + dy * 0.5)
                                        ]
                                    else:
                                        estimated_keypoints[key] = valid_points[-1]

        # 버퍼 업데이트
        self.keypoint_buffer.append(estimated_keypoints)
        self.previous_keypoints = estimated_keypoints

        return estimated_keypoints

    def _validate_leg_chain(self, keypoints, side):
        """
        다리 관절 체인의 유효성을 검증하는 메서드
        
        주요 기능:
        - 다리 관절(엉덩이-무릎-발목) 간의 거리 검증
        - 비현실적인 관절 배치 감지
        - 이전 프레임 데이터를 활용한 보정
        
        검증 항목:
        1. 관절 간 거리의 현실성
        2. 전체 다리 길이의 적절성
        3. 관절 각도의 유효성
        
        Args:
            keypoints (dict): 검증할 키포인트 데이터
            side (str): 검증할 다리 방향 ('left' 또는 'right')
        """
        hip = np.array(keypoints[f'{side}_hip'])
        knee = np.array(keypoints[f'{side}_knee'])
        ankle = np.array(keypoints[f'{side}_ankle'])

        # 관절 간 거리 계산 및 검증
        hip_knee_dist = np.linalg.norm(hip - knee)
        knee_ankle_dist = np.linalg.norm(knee - ankle)
        hip_ankle_dist = np.linalg.norm(hip - ankle)

        # 비현실적인 관절 배치 검사 (전체 길이가 각 부분의 합보다 20% 이상 큰 경우)
        if hip_ankle_dist > (hip_knee_dist + knee_ankle_dist) * 1.2:
            if self.previous_keypoints:
                for joint in [f'{side}_knee', f'{side}_ankle']:
                    if joint in self.previous_keypoints:
                        keypoints[joint] = self.previous_keypoints[joint]

    def normalize_keypoints(self, keypoints):
        """
        포즈 키포인트를 정규화하는 메서드
        
        주요 기능:
        - 키포인트 좌표의 정규화 처리
        - hip center 기준 상대 위치 계산
        - 어깨 너비 기반 스케일 정규화
        - 누락된 데이터 처리
        
        정규화 과정:
        1. 데이터 유효성 검증
        2. hip center 계산 (없을 경우 어깨 중심 사용)
        3. 상대 좌표 변환
        4. 스케일 정규화
        
        Args:
            keypoints (dict): 원본 키포인트 데이터
            
        Returns:
            pd.DataFrame: 정규화된 키포인트 데이터프레임 또는 None (실패 시)
        """
        try:
            # 데이터프레임용 평탄화된 리스트 생성
            kp_flat = []
            all_zeros = True  # 모든 값이 0인지 체크

            # 키포인트 좌표 추출 및 검증
            for name, cols in self.column_mapping.items():
                if name in keypoints:
                    x, y = keypoints[name]
                    if x != 0 or y != 0:
                        all_zeros = False
                    kp_flat.extend([x, y])
                else:
                    kp_flat.extend([0, 0])

            if all_zeros:
                print("경고: 모든 키포인트가 0입니다.")
                return None

            # 데이터프레임 생성
            columns = [col for pair in self.column_mapping.values() for col in pair]
            df = pd.DataFrame([kp_flat], columns=columns)

            # hip center 계산을 위한 검증
            right_hip_x = df['Right_Hip_x'].iloc[0]
            left_hip_x = df['Left_Hip_x'].iloc[0]
            right_hip_y = df['Right_Hip_y'].iloc[0]
            left_hip_y = df['Left_Hip_y'].iloc[0]

            # hip center 계산 (hip이 없는 경우 어깨 중심 사용)
            if (right_hip_x == 0 and left_hip_x == 0) or (right_hip_y == 0 and left_hip_y == 0):
                hip_center_x = (df['Right_Shoulder_x'].iloc[0] + df['Left_Shoulder_x'].iloc[0]) / 2
                hip_center_y = (df['Right_Shoulder_y'].iloc[0] + df['Left_Shoulder_y'].iloc[0]) / 2
            else:
                hip_center_x = (right_hip_x + left_hip_x) / 2
                hip_center_y = (right_hip_y + left_hip_y) / 2

            # hip center 기준 정규화
            for col in df.columns:
                if col.endswith('_x') and df[col].iloc[0] != 0:
                    df[col] = df[col] - hip_center_x
                elif col.endswith('_y') and df[col].iloc[0] != 0:
                    df[col] = df[col] - hip_center_y

            # 어깨 너비 기준 스케일 정규화
            shoulder_width = np.sqrt(
                (df['Right_Shoulder_x'] - df['Left_Shoulder_x']) ** 2 +
                (df['Right_Shoulder_y'] - df['Left_Shoulder_y']) ** 2
            )

            if shoulder_width.iloc[0] > 0:
                for col in df.columns:
                    if (col.endswith('_x') or col.endswith('_y')) and df[col].iloc[0] != 0:
                        df[col] = df[col] / shoulder_width.iloc[0]

            return df

        except Exception as e:
            print(f"키포인트 정규화 중 오류 발생: {str(e)}")
            return None

    def predict(self, image):
        """
        입력 이미지에서 포즈를 예측하는 메인 메서드
        
        주요 기능:
        - 키포인트 추출
        - 데이터 정규화
        - 앙상블 모델을 통한 예측
        - 클래스 매핑 및 결과 반환
        
        처리 단계:
        1. 이미지에서 키포인트 추출
        2. 키포인트 정규화
        3. 모델을 통한 예측 수행
        4. 예측 결과를 클래스 레이블로 변환
        
        Args:
            image (np.ndarray): 입력 이미지
            
        Returns:
            str: 예측된 포즈 클래스명 또는 오류 메시지
        """
        try:
            # 키포인트 추출
            keypoints = self.extract_keypoints(image)
            if keypoints is None:
                return "포즈가 감지되지 않음"

            # 키포인트 정규화
            normalized_kps = self.normalize_keypoints(keypoints)
            if normalized_kps is None:
                return "키포인트 정규화 실패"

            # 모델 입력을 위한 텐서 변환
            inputs = torch.FloatTensor(normalized_kps.values).to(self.device)

            # 각 기본 모델의 예측 수행
            base_predictions = []
            with torch.no_grad():
                for model in self.base_models.values():
                    outputs = model(inputs)
                    probs = F.softmax(outputs, dim=1)
                    base_predictions.append(probs)

                # 스태킹 모델을 통한 최종 예측
                stacked_output = self.stacking_model(inputs)
                final_probs = F.softmax(stacked_output, dim=1)
                predicted_class = final_probs.argmax(dim=1).item()

            return class_mapping[predicted_class]

        except Exception as e:
            print(f"예측 중 오류 발생: {str(e)}")
            return "예측 실패"
        
    def load_base_models(self, model_paths):
        """
        기본 포즈 분류 모델들을 로드하는 메서드
        
        주요 기능:
        - 다양한 아키텍처의 모델 로드 (Transformer, MLP, GRU, CNN, ViT)
        - 체크포인트에서 모델 상태 복원
        - GPU/CPU 디바이스 할당
        - 모델을 평가 모드로 설정
        
        모델 초기화 과정:
        1. 체크포인트 파일 로드
        2. 모델 아키텍처별 인스턴스 생성
        3. 가중치 로드 및 디바이스 할당
        4. 평가 모드 설정
        
        Args:
            model_paths (dict): 모델별 체크포인트 파일 경로
            
        Returns:
            dict: 초기화된 모델들의 딕셔너리
        """
        models = {}
        for model_name, path in model_paths.items():
            if model_name == 'stacking':  # 스태킹 모델은 별도 처리
                continue

            try:
                # 체크포인트 로드
                checkpoint = torch.load(path, map_location=self.device)
                state_dict = checkpoint['model_state_dict']
                
                # 모델 아키텍처별 인스턴스 생성
                if model_name == 'transformer':
                    model = PoseTransformer(input_dim=34, num_classes=20)
                elif model_name == 'mlp':
                    model = PoseMLP()
                elif model_name == 'gru':
                    model = PoseGRU()
                elif model_name == 'cnn':
                    model = PoseCNN()
                else:  # vit
                    model = EnhancedPoseViT()

                # 가중치 로드 및 모델 설정
                model.load_state_dict(state_dict, strict=True)
                model = model.to(self.device)  # GPU/CPU 할당
                model.eval()  # 평가 모드 설정
                models[model_name] = model
                print(f"{model_name} 모델 로드 성공")
                
            except Exception as e:
                print(f"{model_name} 모델 로드 중 오류 발생: {str(e)}")
                continue

        return models

    def load_stacking_model(self, path):
        """
        앙상블 스태킹 모델을 로드하는 메서드
        
        주요 기능:
        - 스태킹 모델 인스턴스 생성
        - 체크포인트에서 가중치 복원
        - GPU/CPU 디바이스 할당
        - 평가 모드 설정
        
        Args:
            path (str): 스태킹 모델 체크포인트 파일 경로
            
        Returns:
            StackingModel: 초기화된 스태킹 모델 또는 None (실패 시)
        """
        try:
            # 스태킹 모델 인스턴스 생성
            model = StackingModel()
            
            # 체크포인트 로드 및 가중치 복원
            checkpoint = torch.load(path, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'], strict=False)
            
            # 디바이스 할당 및 평가 모드 설정
            model = model.to(self.device)
            model.eval()
            return model
            
        except Exception as e:
            print(f"스태킹 모델 로드 중 오류 발생: {str(e)}")
            return None

class RealtimePosePredictor:
    """
    실시간 포즈 예측을 위한 클래스
    
    주요 기능:
    - 실시간 비디오 스트림 처리
    - 멀티스레딩을 통한 성능 최적화
    - CPU 최적화된 처리 파이프라인
    - 한글 텍스트 렌더링
    
    구성 요소:
    - YOLO 객체 탐지기
    - 포즈 예측 모델
    - 프레임 처리 큐
    - FPS 측정 시스템
    """
    def __init__(self, model_paths, yolo_path, yolo_pose_path, font_path):
        """
        실시간 포즈 예측기 초기화
        
        Args:
            model_paths (dict): 각 모델의 체크포인트 파일 경로
            yolo_path (str): YOLO 모델 파일 경로
            yolo_pose_path (str): YOLO 포즈 모델 파일 경로
            font_path (str): 한글 폰트 파일 경로
            
        초기화 과정:
        1. CPU 최적화 설정
        2. YOLO 모델 초기화
        3. 멀티스레딩 설정
        4. 성능 최적화를 위한 큐 설정
        """
        self.device = torch.device('cpu')
        print("CPU 모드로 실행중...")
        
        # 경로 저장
        self.yolo_path = yolo_path
        self.yolo_pose_path = yolo_pose_path
        self.font_path = font_path
        
        # YOLO 모델 초기화 (CPU 최적화)
        try:
            self.yolo = YOLO(self.yolo_path)
            self.yolo_pose = YOLO(self.yolo_pose_path)
            
            # CPU 최적화 설정
            self.yolo.conf = 0.3        # 신뢰도 임계값 조정
            self.yolo_pose.conf = 0.3
            self.yolo.iou = 0.45       # IOU 임계값 조정
            self.yolo_pose.iou = 0.45
            
        except Exception as e:
            print(f"YOLO 모델 초기화 중 오류 발생: {str(e)}")
            raise

    def _process_frames(self):
        """
        프레임 처리를 위한 백그라운드 스레드 메서드
        
        주요 기능:
        - 프레임 큐에서 이미지를 가져와 처리
        - CPU 리소스 최적화
        - 연속적인 프레임 처리
        - 처리된 결과를 결과 큐에 저장
        
        처리 과정:
        1. 프레임 큐에서 이미지 추출
        2. 이미지 크기 조정
        3. 단일 프레임 처리 실행
        4. 결과 큐에 처리된 프레임 저장
        """
        while self.running:
            try:
                if not self.frame_queue.empty():
                    frame = self.frame_queue.get()
                    # 프레임 크기 조정으로 처리 속도 향상
                    frame = cv2.resize(frame, (self.frame_width, self.frame_height))
                    result = self._process_single_frame(frame)
                    if not self.result_queue.full():
                        self.result_queue.put(result)
                else:
                    # CPU 부하 감소를 위한 짧은 대기
                    time.sleep(0.01)
            except Exception as e:
                print(f"프레임 처리 중 오류 발생: {e}")
                continue

    def _process_single_frame(self, frame):
        """
        단일 프레임 처리를 위한 메서드
        
        주요 기능:
        - 여러 사람의 포즈 동시 감지
        - 키포인트 추출 및 시각화
        - 바운딩 박스 및 스켈레톤 그리기
        - FPS 정보 표시
        
        처리 단계:
        1. 입력 프레임 검증
        2. 포즈 감지 및 키포인트 추출
        3. 시각적 요소 추가 (스켈레톤, 바운딩 박스 등)
        4. FPS 계산 및 표시
        
        Args:
            frame (np.ndarray): 처리할 입력 프레임
            
        Returns:
            np.ndarray: 처리된 프레임
        """
        try:
            if frame is None or not isinstance(frame, np.ndarray):
                return frame

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # 여러 사람의 키포인트 추출
            all_keypoints = self.pose_predictor.extract_keypoints(frame)
            
            if all_keypoints:
                for person_keypoints in all_keypoints:
                    # 바운딩 박스 그리기
                    if 'bbox' in person_keypoints:
                        x1, y1, x2, y2, conf = person_keypoints['bbox']
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                                    (0, 255, 0), 2)
                        cv2.putText(frame, f'Person {conf:.2f}', (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    try:
                        # 키포인트 정규화 및 포즈 예측
                        normalized_kps = self.pose_predictor.normalize_keypoints(person_keypoints)
                        if normalized_kps is not None:
                            pose_prediction = self.pose_predictor.predict(frame)
                            
                            # 키포인트 시각화
                            for name, point in person_keypoints.items():
                                if name != 'bbox':  # 바운딩 박스 정보 제외
                                    color = self.pose_predictor.keypoint_colors.get(name, (0, 255, 0))
                                    cv2.circle(frame, tuple(point), 5, color, -1)

                            # 스켈레톤 연결선 그리기
                            for start, end in self.pose_predictor.skeleton_connections:
                                if start in person_keypoints and end in person_keypoints:
                                    start_point = tuple(person_keypoints[start])
                                    end_point = tuple(person_keypoints[end])
                                    
                                    is_upper_body = all(point in ['left_shoulder', 'right_shoulder', 
                                                                'left_elbow', 'right_elbow',
                                                                'left_wrist', 'right_wrist'] 
                                                    for point in [start, end])
                                    color = (255, 0, 0) if is_upper_body else (0, 0, 255)
                                    cv2.line(frame, start_point, end_point, color, 2)

                            # 예측 결과 표시
                            if 'bbox' in person_keypoints:
                                x1, y1 = person_keypoints['bbox'][:2]
                                cv2.putText(frame, str(pose_prediction), 
                                        (int(x1), int(y1) - 30),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    except Exception as e:
                        print(f"개별 사람 처리 중 오류 발생: {e}")

                # FPS 표시
                current_time = time.time()
                fps = 1.0 / (current_time - self.prev_time)
                self.prev_time = current_time
                cv2.putText(frame, f"FPS: {fps:.1f}", (10, frame.shape[0] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            return frame

        except Exception as e:
            print(f"프레임 처리 중 오류 발생: {e}")
            return frame
        
    def draw_text_with_korean(self, img, text, position, font_color=(255, 255, 255)):
        """
        한글 텍스트를 이미지에 그리는 메서드
        
        주요 기능:
        - PIL을 사용한 한글 텍스트 렌더링
        - OpenCV 이미지와 PIL 이미지 간 변환
        - 사용자 지정 폰트 및 색상 지원
        
        Args:
            img (np.ndarray): OpenCV 형식의 이미지
            text (str): 그릴 텍스트 (한글 지원)
            position (tuple): 텍스트를 그릴 위치 (x, y)
            font_color (tuple): RGB 형식의 폰트 색상 (기본값: 흰색)
            
        Returns:
            np.ndarray: 텍스트가 그려진 이미지
        """
        try:
            img_pil = Image.fromarray(img)  # OpenCV -> PIL 변환
            draw = ImageDraw.Draw(img_pil)  # 드로잉 객체 생성
            draw.text(position, text, font=self.font, fill=font_color)
            return np.array(img_pil)  # PIL -> OpenCV 변환
        except Exception as e:
            print(f"텍스트 그리기 중 오류 발생: {e}")
            return img

    def process_frame(self, frame):
        """
        프레임 처리를 위한 메인 인터페이스 메서드
        
        주요 기능:
        - 프레임 스킵을 통한 성능 최적화
        - FPS 측정 및 관리
        - 프레임 처리 큐 관리
        - 이전 프레임 캐싱
        
        처리 단계:
        1. 프레임 스킵 로직 적용
        2. FPS 업데이트
        3. 프레임 처리 큐 관리
        4. 결과 반환
        
        Args:
            frame (np.ndarray): 처리할 입력 프레임
            
        Returns:
            np.ndarray: 처리된 프레임 또는 이전 프레임
        """
        try:
            self.frame_count += 1

            # 프레임 스킵 로직
            if self.frame_count % self.frame_skip != 0:
                return self.previous_frame if self.previous_frame is not None else frame

            # FPS 업데이트
            current_time = time.time()
            self.fps_deque.append(current_time - self.prev_time)
            self.prev_time = current_time

            # 입력 검증
            if not isinstance(frame, np.ndarray):
                frame = np.array(frame)

            # 프레임 처리 큐에 추가
            if not self.frame_queue.full():
                self.frame_queue.put(frame)

            # 처리된 결과 가져오기
            if not self.result_queue.empty():
                self.previous_frame = self.result_queue.get()
                return self.previous_frame

            return self.previous_frame if self.previous_frame is not None else frame

        except Exception as e:
            print(f"프레임 처리 중 오류 발생: {e}")
            return frame

    def run(self):
        """
        실시간 포즈 감지 시스템의 메인 실행 루프
        
        주요 기능:
        - 웹캠 초기화 및 설정
        - 실시간 프레임 처리
        - 사용자 입력 처리
        - 리소스 정리
        
        실행 과정:
        1. 웹캠 초기화 및 설정
        2. 프레임 캡처 및 처리
        3. 결과 표시
        4. 종료 처리
        """
        print("웹캠 초기화 중...")
        cap = cv2.VideoCapture(0)
        
        # 웹캠 설정 (저해상도 설정으로 성능 최적화)
        cap.set(cv2.CAP_PROP_FPS, 15)  # FPS 제한
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.frame_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.frame_height)
        
        print("웹캠 설정 완료")
        print(f"해상도: {self.frame_width}x{self.frame_height}")
        print(f"목표 FPS: 15")

        try:
            while cap.isOpened() and self.running:
                ret, frame = cap.read()
                if not ret:
                    print("카메라로부터 프레임을 가져오는데 실패했습니다.")
                    break

                # 프레임 처리 로직
                self.frame_count += 1
                if self.frame_count % self.frame_skip != 0:
                    continue

                # 프레임 처리 큐 관리
                if not self.frame_queue.full():
                    self.frame_queue.put(frame)

                # 결과 표시
                if not self.result_queue.empty():
                    processed_frame = self.result_queue.get()
                    if processed_frame is not None:
                        cv2.imshow('실시간 포즈 감지', processed_frame)

                # 키 입력 처리
                key = cv2.waitKey(1) & 0xFF
                if key in [27, ord('q')]:  # ESC 또는 'q' 키
                    break

        except Exception as e:
            print(f"실행 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
        finally:
            # 리소스 정리
            print("프로그램 종료 중...")
            self.running = False
            if cap.isOpened():
                cap.release()
            cv2.destroyAllWindows()

def main():
    """
    포즈 감지 시스템의 메인 실행 함수
    
    주요 기능:
    - 시스템 환경 확인 및 초기화
    - GPU/CPU 설정
    - 모델 및 리소스 파일 경로 설정
    - 실시간 포즈 감지 시스템 실행
    
    초기화 과정:
    1. 시스템 정보 출력
    2. GPU 사용 가능 여부 확인
    3. 필요한 모델 및 리소스 파일 검증
    4. 포즈 감지 시스템 실행
    """
    try:
        # 시스템 정보 출력
        print("시스템 정보:")
        print(f"Python 버전: {sys.version}")
        print(f"OpenCV 버전: {cv2.__version__}")
        print(f"PyTorch 버전: {torch.__version__}")
        print(f"CPU 스레드 수: {os.cpu_count()}")
        
        # GPU 사용 가능 여부 확인
        if not torch.cuda.is_available():
            print("GPU를 사용할 수 없어 CPU를 사용합니다")
            device = torch.device('cpu')
        else:
            print("GPU를 사용합니다")
            device = torch.device('cuda')

        # 기본 디렉토리 설정
        BASE_DIR = r"C:\coding\python\vs\새 폴더"
        
        # 모델 경로 설정
        model_paths = {
            'transformer': os.path.join(BASE_DIR, 'transformer_complete.pth'),
            'mlp': os.path.join(BASE_DIR, 'mlp_complete.pth'),
            'gru': os.path.join(BASE_DIR, 'gru_complete.pth'),
            'cnn': os.path.join(BASE_DIR, 'cnn_complete.pth'),
            'vit': os.path.join(BASE_DIR, 'vit_complete.pth'),
            'stacking': os.path.join(BASE_DIR, 'stacking_complete.pth')
        }

        # YOLO 모델 및 폰트 파일 경로 설정
        yolo_path = os.path.join(BASE_DIR, 'yolov8n.pt')        # 경량화된 모델
        yolo_pose_path = os.path.join(BASE_DIR, 'yolov8x.pt')   # 포즈 추정 모델
        font_path = r"C:\Users\user\Desktop\vision\GowunDodum-Regular.ttf"

        # 필요한 파일 존재 여부 확인
        for name, path in model_paths.items():
            if not os.path.exists(path):
                print(f"경고: {name} 모델 파일이 없습니다: {path}")

        # 필수 파일 검증
        if not os.path.exists(yolo_path):
            raise FileNotFoundError(f"YOLO 모델 파일이 없습니다: {yolo_path}")
        
        if not os.path.exists(yolo_pose_path):
            raise FileNotFoundError(f"YOLO Pose 모델 파일이 없습니다: {yolo_pose_path}")
            
        if not os.path.exists(font_path):
            raise FileNotFoundError(f"폰트 파일이 없습니다: {font_path}")

        print("모든 필요 파일 확인 완료")
        print("프로그램 시작 중...")

        # PyTorch 성능 최적화 설정
        torch.set_num_threads(4)  # CPU 스레드 수 제한
        
        # 실시간 포즈 예측기 초기화 및 실행
        predictor = RealtimePosePredictor(model_paths, yolo_path, yolo_pose_path, font_path)
        predictor.run()

    except Exception as e:
        # 예외 처리 및 상세 에러 로그 출력
        print(f"프로그램 실행 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    """
    프로그램 시작점
    
    - 프로그램이 직접 실행될 때만 main() 함수 호출
    - 모듈로 임포트될 때는 실행되지 않음
    """
    main()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3540435477.py, line 515)

In [18]:
py -3.10 -m pip install torch torchvision torchaudio

SyntaxError: invalid syntax (2203347687.py, line 1)

In [15]:
pip install torch.nn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch.nn (from versions: none)
ERROR: No matching distribution found for torch.nn

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
pip install python


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python
